# Exercise 1: Recommender System from scratch

 To build a recommendation engine using matrix factorization, we read the data in an RDD and split the data into test and train using random split function.

In [1]:
from pyspark import SparkContext,SparkConf
from pyspark import SQLContext
import pandas as pd

sc = SparkContext()
sqlContext = SQLContext(sc)

#Read data
rating = pd.read_csv(r'/home/kritz/Documents/DDL/Ex10/movieLens/ratings.csv')
ratings = sqlContext.createDataFrame(rating)

#train test split
Train,test = ratings.rdd.randomSplit([0.8, 0.2])

To intialize matrix, we get the number of users and movies by getting the maximum of available values

In [2]:
#Size u and size v 
sizeU = Train.map(lambda x: int(x[0])).max() + 1
sizeV = Train.map(lambda x: int(x[1])).max() + 1 

print(sizeU,sizeV)

672 163950


Next we have to build a matrix with all the userId, movieId and actual ratings, we get the individual values and convert them to a matrix, we use a sparse matrix as the ratings are sparse in nature, a user does not rate all the available movies,a csr matrix is more efficient way

In [3]:
from scipy.sparse import csr_matrix

userId = Train.map(lambda x: int(x[0])).collect()
movieId = Train.map(lambda x: int(x[1])).collect()
rate = Train.map(lambda x: int(x[2])).collect()  

#Getting matrix UV
UV = csr_matrix((rate,(userId,movieId)), shape=(sizeU,sizeV))
print(UV[1])

  (0, 31)	2
  (0, 1029)	3
  (0, 1061)	3
  (0, 1129)	2
  (0, 1172)	4
  (0, 1263)	2
  (0, 1287)	2
  (0, 1293)	2
  (0, 1343)	2
  (0, 1371)	2
  (0, 1405)	1
  (0, 1953)	4
  (0, 2105)	4
  (0, 2150)	3
  (0, 2193)	2
  (0, 2455)	2


Repeating the same process for test data to get a test UV matrix

In [4]:
userId = test.map(lambda x: int(x[0])).collect()
movieId = test.map(lambda x: int(x[1])).collect()
rate = test.map(lambda x: int(x[2])).collect()  

#Getting matrix UV
uvTest = csr_matrix((rate,(userId,movieId)), shape=(sizeU,sizeV))
print(uvTest[1])

  (0, 1339)	3
  (0, 2294)	2
  (0, 2968)	1
  (0, 3671)	3


Next, we have to initialize the U and V matrices to random values to learn the matrix factorization with K latent factors

In [ ]:
import numpy as np

#Latent factors
K = 20
#Partititons
P = 10

#Initialize matrix
uLatent = np.random.random_sample((sizeU,K))
vLatent = np.random.random_sample((K,sizeV))

#Partition train data
trainParts = Train.repartition(P).persist()
trainParts.collect

We use RMSE Loss the function returns the RMSE

In [8]:
def trainLoss(UV,mat):
    mse = mean_squared_error(UV.todense(),mat)
    return np.sqrt(mse)

def testLoss(test,mat):
    rmse =mean_squared_error(test.collect(),mat)
    return np.sqrt(mse)

The SGD function loops the iterations, it builds a smaller UV matrix for each partition in the data. This is used to update the uLatent nd vLatent matrices seperatly. These matrices are updated using the gradient of the loss function and returned to the main function where its average is taken

In [9]:
def SGD(partition,uLatent,vLatent,iters,alpha):
    
    userId,movieId,rating = [],[],[]
    
    for element in partition:
        userId.append(element[0])   
        movieId.append(element[1]) 
        rating.append(element[2])   
        
    users = np.amax(userId) +1   
    movies = np.amax(movieId) +1 
    
    #create UV matrix with userIds and moviesIds present in partition
    partUV = csr_matrix((rating,(userId,movieId)), shape=(users,movies))
                                                                .todense()
    
    #non-zero elements
    rows,cols = partUV.nonzero()
    i = 0  
    
    #SGD
    while i < iters:
        #Computing gradient for a random sample
        randomNum = random.randint(0,(rows).size-1)  
        r = rows[randomNum]   
        c = cols[randomNum]  
        sample = partUV[r,c] 

        #compute gradients for U and V
        temp1 = -2*(sample - uLatent[r,:].dot(vLatent[:,c]))*(vLatent[:,c].T)
        uGrad = temp1 + (2*alpha*uLatent[r,:])/(partUV[r,:].nonzero()[0].size)
        temp2 = -2*(sample - uLatent[r,:].dot(vLatent[:,c]))*(uLatent[r,:].T)
        vGrad = temp2 + (2*alpha*vLatent[:,c])/(partUV[:,c].nonzero()[0].size)

        #update the U and V values
        uLatent[r,:] = uLatent[r,:] - alpha*uGrad 
        vLatent[:,c] = vLatent[:,c] - alpha*vGrad
        i += 1    

    return uLatent,vLatent

The training function is where the data is sent to the SGD function, once all the part uLatent and vLatent matrices are calculated, their average is taken to get the final updated matrix. Persist is used to avoid memory errors

In [10]:
def training(trainParts,alpha,uLatent,vLatent,UV,test):
    #Training
    #Learning rate
    iters,epoch = 20,5  
    i = 0
    trainRmse,testRmse = [],[]

    while i < epoch: 
        print(i)
        updatedUV = trainParts.mapPartitions(lambda ratings_part: 
                        SGD(ratings_part,uLatent,vLatent,iters,alpha)).persist()
        newUV = updatedUV.zipWithIndex().map(lambda x: (x[1]%2, x[0])
                .reduceByKey(lambda r, k: r+k).map(lambda q: (q[1]/P)).persist()

        #update uLatent
        uLatent = newUV.collect()[0]

        #update vLatent
        vLatent = newUV.collect()[1]

        #prediction
        prediction = np.matmul(uLatent,vLatent)

        #TrainLoss
        TrainLoss= trainLoss(UV,prediction)
        
        #Validation loss
        TestLoss = testLoss(test,prediction)
        trainRmse.append(TrainLoss)
        testRmse.append(TestLoss)
        i = i+1
    return(np.mean(testRmseLoss))

The cross-validation function is where the training data is split into 3 folds and all their combinations are trained, only two values of alpha are considered as the computation time is higher and efficiency. The alpha with lowest validation loss is selected.

In [15]:
def crossValidation(alpha,trainOrig):
    totLoss = []
    #Splitting data into folds
    parts = trainOrig.randomSplit([0.33, 0.33,0.34])
    #Possible train and test combinations
    trainSeq = [[0,1],[0,2],[1,2]]
    testSeq = [2,1,0]
    for i in range(0,3):
        train = sc.union([parts[trainSeq[i][0]],parts[trainSeq[i][1]]])
        Test = parts[testSeq[i]]
    
        #Size u and size v 
        sizeU = train.map(lambda x: int(x[0])).max() + 1
        sizeV = train.map(lambda x: int(x[1])).max() + 1 

        from scipy.sparse import csr_matrix

        userId = train.map(lambda x: int(x[0])).collect()
        movieId = train.map(lambda x: int(x[1])).collect()
        rate = train.map(lambda x: int(x[2])).collect()  

        #Getting matrix UV
        UV = csr_matrix((rate,(userId,movieId)), shape=(sizeU,sizeV))

        userId1 = Test.map(lambda x: int(x[0])).collect()
        movieId1 = Test.map(lambda x: int(x[1])).collect()
        rate1 = Test.map(lambda x: int(x[2])).collect()  

        #Getting matrix UV
        uvTest = csr_matrix((rate1,(userId1,movieId1)), shape=(sizeU,sizeV))
        #print(uvTest[1])

        #Latent factors
        K = 20
        #Partititons
        P = 10

        #Initialize matrix
        uLatent = np.random.random_sample((sizeU,K))
        vLatent = np.random.random_sample((K,sizeV))

        #Partition train data
        trainParts = train.repartition(P).persist()
        trainParts.collect
        
        loss = training(trainParts,alpha,uLatent,vLatent,UV,Test)
        #print("loss",loss)
        totLoss.append(loss)
    return (np.mean(totLoss))

In [20]:
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import numpy as np
import random 

alphaGrid = [0.1,0.2]
for each,loss in zip(alphaGrid,bongu):
    check = []
    P = 10
    print("Alpha = ", each)
    loss = crossValidation(each,Train)
    print("loss = ",loss)
    check.append(loss)

Alpha =  0.1
loss =  5.063878816619068
Alpha =  0.2
loss =  5.03456525902943


It can be seen that for alpha = 0.2 the loss is less, so this parameter is selected, executing the previous statements the train and test loss are

Train RMSE = 4.4563827615429825
Test RMSE = 3.7691529816539208